In [1]:
import cv2, numpy, time
from imutils.video import WebcamVideoStream, FPS
%matplotlib inline
import matplotlib as plt

In [5]:
cap = cv2.VideoCapture(0)
for i in range(10000000):
    r = False
    while not r:
        r, img = cap.read()
    bw = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
    blr = cv2.medianBlur(img,5)
    th3 = cv2.adaptiveThreshold(blr, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    cv2.imshow("input", img)
    cv2.imshow("output", th3)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break
cap.release()

error: OpenCV(3.4.1) C:\bld\opencv_1520732670222\work\opencv-3.4.1\modules\imgproc\src\thresh.cpp:1524: error: (-215) src.type() == (((0) & ((1 << 3) - 1)) + (((1)-1) << 3)) in function cv::adaptiveThreshold
